In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd 
import numpy as np

In [2]:
# load the train model, scaler pickle, onehot
model = load_model('model.h5')

# load  the encoder and scaler 
with open('onehot_encoder_geo.pkl','rb') as file :
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file :
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file :
    scaler = pickle.load(file)

In [10]:
# example input data
input_data = {
    'CreditScore' : 600, 
    'Geography' : 'Germany', 
    'Gender' : "Male", 
    'Age' : 40, 
    'Tenure' : 3, 
    'Balance' : 50000,
    'NumOfProducts' : 3, 
    'HasCrCard' : 1, 
    'IsActiveMember' : 1, 
    'EstimatedSalary' : 70000
}

In [11]:
# onehot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns = label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df 

C:\Users\DELL\.conda\envs\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [12]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Germany,Male,40,3,50000,3,1,1,70000


In [14]:
# label categorical variable
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Germany,1,40,3,50000,3,1,1,70000


In [16]:
input_df  = pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,50000,3,1,1,70000,0.0,1.0,0.0


In [17]:
## scaling the input df
input_scaled =scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.41792108,
         2.53355998,  0.64920267,  0.97481699, -0.5290988 , -0.99850112,
         1.72572313, -0.57638802]])

#### Predictions 

In [19]:
## predict churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


array([[0.49129957]], dtype=float32)

In [20]:
prediction_prob = prediction[0][0]

In [21]:
prediction_prob

0.49129957

In [22]:
if prediction_prob > 0.5 :
    print('The customer is likely to churn.')
else :
    print('The customer is not likely to churn.')

The customer is not likely to churn.
